In [ ]:
import scipy
import plotly.graph_objects as go 
import mne
from gtda.diagrams import Scaler, BettiCurve
from sklearn.decomposition import PCA
from gtda.time_series import TakensEmbedding
from gtda.time_series import takens_embedding_optimal_parameters, SlidingWindow
from gtda.plotting import plot_point_cloud
import plotly.graph_objects as go 
import numpy as np
from gtda.homology import VietorisRipsPersistence, WeakAlphaPersistence
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, plot,iplot

In [ ]:
def get_age(num_subject):
    return np.unique(sheetX[(sheetX.subject == num_subject)]['age'].tolist())[0]
def get_time(num_subject, num_night):
    return np.unique(sheetX[(sheetX.subject == num_subject) & (sheetX.night == num_night)]['LightsOff'].tolist())[0]
def get_gender(num_subject):
    return np.unique(sheetX[(sheetX.subject == num_subject)]["sex (F=1)"].tolist())[0]

In [ ]:
from gtda.diagrams import PersistenceLandscape
def get_land(n, name):
  PL = PersistenceLandscape(n_layers=1, n_bins=100, n_jobs = -1)
  temp = PL.fit_transform(get_diag(n, name))
  fig = PL.plot(temp, homology_dimensions=[1])
  av = PL.plot(temp, homology_dimensions=[0])
  fig.add_trace(av['data'][0], row=1, col=1)
  fig.show()
  #fig = PI.plot(temp, homology_dimensions=1)
  #fig.show()
  return temp
  #fig.write_image(f'1.pdf') 
  #plt.show()

In [ ]:
from gtda.diagrams import PersistenceImage
def get_image(aw):
  PI = PersistenceImage(sigma=0.03, n_bins=500, n_jobs = -1)
  temp = PI.fit_transform(aw)
  fig = PI.plot(temp, homology_dimension_idx=0)

  fig.show()
  fig = PI.plot(temp, homology_dimension_idx=1)
  fig.show()
  return temp
  #fig.write_image(f'1.pdf') 
  #plt.show()

In [ ]:
def get_diag(raw_n, n, name): 
  print(name)
  y = raw_n['EEG Fpz-Cz'][1][n * 100 :n * 100 + 100 * 30 + 1]
  x = raw_n['EEG Fpz-Cz'][0][0][n * 100:n * 100 + 100 * 30 + 1]
  data = np.hstack((np.array(x), np.array(y)))
  optimal_time_delay,optimal_embedding_dimension = takens_embedding_optimal_parameters(data, 300, 5, stride=1) 
  TE = TakensEmbedding(
    time_delay=optimal_time_delay,
    dimension=optimal_embedding_dimension,
    stride = 1
  )
  window_size = 3000 

  window_stride = 2 

  SW = SlidingWindow(size=window_size, stride=window_stride) 

  X_windows, y_windows = SW.fit_transform_resample(x, y) 
  homology_dimensions = [0, 1]
  y_nonperiodic_embedded = TE.fit_transform(X_windows)
  WA = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=-1
  )
  aw = WA.fit_transform(y_nonperiodic_embedded)
  
  if aw[0][len(aw) - 1][0] == aw[0][len(aw) - 1][1]:
        aw[0] = aw[0][:len(aw) - 1]

  diagramScaler = Scaler()
  X_scaled = diagramScaler.fit_transform(aw)
  return X_scaled

In [ ]:
ST_cop = np.zeros((100000, 34))
STAT = np.zeros((100000, 34))
from scipy.signal import find_peaks
def get_statistics(raw_n, n, aw, name, j):
  M1 = []
  M0 = []
  L1 = []
  L0 = []
  for i in range(len(aw[0])):
    if aw[0][i][2] == 1.:
      if abs(aw[0][i][0] - aw[0][i][1]) < 0.005:
        continue
      M1.append((aw[0][i][0] + aw[0][i][1]) / 2)
      L1.append(aw[0][i][1] - aw[0][i][0])
    else:
      if abs(aw[0][i][0] - aw[0][i][1]) < 0.005:
        continue
      M0.append((aw[0][i][0] + aw[0][i][1]) / 2)
      L0.append(aw[0][i][1] - aw[0][i][0])
  ST = np.vstack((np.array(M0),np.array(L0)))
  ST1 = np.vstack((np.array(M1), np.array(L1)))
  #print(ST)
  #print(ST1)
  k = 0
  for i in range(2):
    arr = np.array(ST[i])
    summ = np.sum(arr)
    STAT[j][0 + k] =np.mean(arr)
    STAT[j][1 + k] =np.std(arr)
    STAT[j][4 + k] = np.quantile(arr, 0.25, axis=0)
    STAT[j][5 + k] = np.quantile(arr, 0.5, axis=0)
    STAT[j][6 + k] = np.quantile(arr, 0.75, axis=0)
    STAT[j][2 + k] = scipy.stats.skew(arr)
    STAT[j][3 + k] = scipy.stats.kurtosis(arr)
    STAT[j][7 + k] = np.sum((-arr/summ) * np.log(arr/summ))
    k = 8
  k += 8
  for i in range(2):
    arr = np.array(ST1[i])
    #print(arr)
    summ = np.sum(arr)
    STAT[j][0 + k] =np.mean(arr)
    STAT[j][1 + k] =np.std(arr)
    STAT[j][4 + k] = np.quantile(arr, 0.25, axis=0)
    STAT[j][5 + k] = np.quantile(arr, 0.5, axis=0)
    STAT[j][6 + k] = np.quantile(arr, 0.75, axis=0)
    STAT[j][2 + k] = scipy.stats.skew(arr)
    STAT[j][3 + k] = scipy.stats.kurtosis(arr)
    STAT[j][7 + k] = np.sum((-arr/summ) * np.log(arr/summ))
    k += 8
  
  yf = np.fft.fft(raw_n['EEG Fpz-Cz'][0][0][n * 100 :n * 100 + 100 * 30 + 1]).real
  peaks_index, properties = find_peaks(np.abs(yf), height=0.005)
  for i in range(len(peaks_index)):
    if (i % 2 == 0):
      if i + 1 < len(peaks_index):
        STAT[j][33] = max(STAT[j][33], peaks_index[i + 1] - peaks_index[i])
  STAT[j][32] = name
  ST_cop = STAT.copy()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
PATH = "Datasets/sleep-edf-database-expanded-1.0.0/sleep-cassette/"
subjects = [i for i in range(83)]
train, test = train_test_split(subjects, test_size = 0.3, random_state=42)
xls = pd.ExcelFile(r"Datasets/sleep-edf-database-expanded-1.0.0/SC-subjects.xls")
sheetX = xls.parse(0)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
PATH = "Datasets/sleep-edf-database-expanded-1.0.0/sleep-cassette/"
subjects = [i for i in range(83)]
train, test = train_test_split(subjects, test_size = 0.3, random_state=23)
xls = pd.ExcelFile(r"Datasets/sleep-edf-database-expanded-1.0.0/SC-subjects.xls")
sheetX = xls.parse(0)

for i in test:
    for j in range(1, 3):
        print(i, j)
        if i == 13 and j == 2:
            break
        num_sub = str(i)
        if i < 10:
            num_sub = '0' + str(i)
        prefixed = [filename for filename in os.listdir(PATH) if filename.startswith("SC4" + num_sub + str(j))]
        raw = mne.io.read_raw_edf(PATH + prefixed[0], preload=True) 
        annot = mne.read_annotations(PATH + prefixed[1])
        annotation_desc_2_event_id = {'Sleep stage W': 1, 'Sleep stage 1': 2, 'Sleep stage 2': 3,'Sleep stage 3': 4,'Sleep stage 4': 5, 'Sleep stage R': 6}
        if get_time(i,j).hour > 8:
            annot.crop(annot[1]['onset'] - 30 * 60, annot[-2]['onset'] + 30 * 60)
        raw_n = raw.copy()
        raw_filt = raw_n.copy().filter(0.1, 30)
        raw_n = raw_filt.copy() 
        event_id = {'Sleep stage W': 1,'Sleep stage 1': 2,'Sleep stage 2': 3,'Sleep stage 3': 4,'Sleep stage 4': 5,'Sleep stage R': 6} 
        raw_n.set_annotations(annot, emit_warning=False) 
        events, _ = mne.events_from_annotations(raw_n, event_id=annotation_desc_2_event_id, chunk_duration=30.)
        #fig = mne.viz.plot_events(events, event_id=event_id,sfreq=raw.info['sfreq'],first_samp=events[0, 0], on_missing='ignore')
        #stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color'] 
        #plt.savefig('Stages_' + str(i) + '_' + str(j) + '.png')
        tmax = 30. - 1. / raw_n.info['sfreq']
        epochs_train = mne.Epochs(raw=raw_n, events=events,
                                  event_id=event_id, tmin=0., tmax=tmax, baseline=None, on_missing='ignore')
        id_e = 0
        for event_id in events:
            ans = get_diag(raw_n, event_id[0] // 100, event_id[2])
            #print('ans: ', ans)
            get_statistics(raw_n, event_id[0] // 100, ans, event_id[2], id_e)
            id_e += 1

In [ ]:
STAT = np.copy(DO_NOT_TOUCH[:1000])
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
def get_data():
    dataset = pd.DataFrame({'STAGE': STAT[:, 32], 'MEAN_M0': STAT[:, 0], 'STD_M0': STAT[:, 1], 'stew_M0': STAT[:, 2], 'kurtosis_M0': STAT[:, 3],
                            '25_M0': STAT[:, 4], '50_M0': STAT[:, 5], '75_M0': STAT[:, 6], 'ENT_M0': STAT[:, 7],
                            'MEAN_L0': STAT[:, 8], 'STD_L0': STAT[:, 9], 'stew_L0': STAT[:, 10], 'kurtosis_L0': STAT[:, 11],
                            '25_L0': STAT[:, 12], '50_L0': STAT[:, 13], '75_L0': STAT[:, 14], 'ENT_L0': STAT[:, 15],
                            'MEAN_M1': STAT[:, 16], 'STD_M1': STAT[:, 17], 'stew_M1': STAT[:, 18], 'kurtosis_M1': STAT[:, 19],
                            '25_M1': STAT[:, 20], '50_M1': STAT[:,21], '75_M1': STAT[:, 22], 'ENT_M1': STAT[:, 23],
                            'MEAN_L1': STAT[:, 24], 'STD_L1': STAT[:, 25], 'stew_L1': STAT[:, 26], 'kurtosis_L1': STAT[:, 27],
                            '25_L1': STAT[:, 28], '50_L1': STAT[:, 29], '75_L1': STAT[:, 30], 'ENT_L1': STAT[:, 31],
                          'FFT': STAT[:, 33]})
    return dataset
dataset = get_data()
from sklearn.svm import LinearSVC
y = dataset['STAGE']
y = label_binarize(y, classes=[1, 2, 3, 4, 5, 6])
n_classes = y.shape[1]
X = dataset.drop(columns='STAGE')
clf = OneVsRestClassifier(
    svm.SVC(kernel="linear", probability=True, random_state=0)
)
y_score = (clf.fit(X, y).decision_function(X))
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])